# 🌍 Módulo 01b - Extracción y Transformación EUROSTAT

**Nombre del archivo:** `01b_extract_transform_EUROSTAT.ipynb`  
**Objetivo:** Extraer **14 DataFrames** de indicadores de desigualdad desde la API de EUROSTAT (SDMX-JSON)  
**Fuentes de datos:** EUROSTAT - EU-SILC (Statistics on Income and Living Conditions), datasets: ilc_di12, ilc_li02, ilc_di11, sdg_10_30  
**Fecha de última edición:** 2025-11-16  
**Autor:** Mario (databamario)  

**Nota:** Consulta [`docs/DICCIONARIO_DATOS.md`](../../docs/DICCIONARIO_DATOS.md) para documentación completa de variables y metodología.

---

## 📊 Indicadores Extraídos

In [1]:
import requests
import pandas as pd
import pickle
from pathlib import Path


# Compute project_root robustly and setup cache dir
def _find_project_root():
    p = Path.cwd()
    while p != p.parent:
        if (p / ".git").exists() or (p / "README.md").exists():
            return p
        p = p.parent
    return Path.cwd()


project_root = _find_project_root()
CACHE_DIR = project_root / "outputs" / "pickle_cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Configuración lista. Cache: {CACHE_DIR.absolute()}")

✅ Configuración lista. Cache: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\outputs\pickle_cache


In [2]:
def parsear_eurostat_sdmx(
    data_json,
    value_name,
    filter_geo="ES",
    filter_unit=None,
    filter_indic=None,
    filter_age="TOTAL",
    filter_sex="T",
):
    """
    Parsea la respuesta SDMX-JSON de Eurostat (formato 'compacto') a un DataFrame.
    Filtra por geografía y otras dimensiones opcionales.

    IMPORTANTE: Solo extrae dimensiones que REALMENTE existen en el JSON.
    - ilc_di12b/c (agregadas): Solo [freq, indic_il, geo, time] → NO extrae age/sex
    - ilc_li02 (desagregadas): [freq, unit, indic_il, sex, age, geo, time] → SÍ extrae age/sex

    Parámetros:
    - filter_geo: código de geografía (ej: 'ES', 'EU27_2020', None para todos)
    - filter_unit: unidad (ej: 'PC' para porcentaje, None para ignorar)
    - filter_indic: indicador específico (ej: 'LI_R_MD60', None para ignorar)
    - filter_age: rango de edad (default 'TOTAL', None para ignorar)
    - filter_sex: sexo (default 'T' para Total, None para ignorar)
    """
    try:
        dimensions = data_json.get("dimension", {})
        size = data_json.get("size", [])
        values = data_json.get("value", {})

        # Crear mapeos de índices a valores
        dim_maps = {}
        dim_labels = {}
        dim_keys = list(dimensions.keys())

        # CRÍTICO: Detectar qué dimensiones realmente existen en esta respuesta
        dimensions_exist = set(dim_keys)

        for key in dim_keys:
            idx_map = dimensions.get(key, {}).get("category", {}).get("index", {})
            dim_maps[key] = {v: k for k, v in idx_map.items()}  # Invertido
            dim_labels[key] = (
                dimensions.get(key, {}).get("category", {}).get("label", {})
            )

        records = []
        for key_str, value in values.items():
            if value is None:
                continue

            key = int(key_str)
            indices = []
            for s in reversed(size):
                indices.append(key % s)
                key //= s
            indices.reverse()

            record = {value_name: float(value)}
            valid_record = True

            for i, dim_key in enumerate(dim_keys):
                dim_idx = indices[i]
                code = dim_maps.get(dim_key, {}).get(dim_idx)
                if code is None:
                    valid_record = False
                    break

                # Aplicar filtros específicos por dimensión
                if dim_key == "geo":
                    if filter_geo and code != filter_geo:
                        valid_record = False
                        break
                    record["geo_code"] = code
                    record["geo_name"] = dim_labels.get("geo", {}).get(code, code)

                if dim_key == "time":
                    record["Anio"] = int(code)

                # Filtrar por unidad (si existe en esta dimensión)
                if dim_key == "unit":
                    if filter_unit and code != filter_unit:
                        valid_record = False
                        break

                # Filtrar por indicador (si existe en esta dimensión)
                if dim_key == "indic_il":
                    if filter_indic and code != filter_indic:
                        valid_record = False
                        break

                # ✅ SOLO extraer age si REALMENTE existe en esta dimensión
                if dim_key == "age" and "age" in dimensions_exist:
                    record["age"] = code
                    record["age_label"] = dim_labels.get("age", {}).get(code, code)
                    # Filtrar si se especificó
                    if filter_age is not None and code != filter_age:
                        valid_record = False
                        break

                # ✅ SOLO extraer sex si REALMENTE existe en esta dimensión
                if dim_key == "sex" and "sex" in dimensions_exist:
                    record["sex"] = code
                    record["sex_label"] = dim_labels.get("sex", {}).get(code, code)
                    # Filtrar si se especificó
                    if filter_sex is not None and code != filter_sex:
                        valid_record = False
                        break

            if valid_record:
                records.append(record)

        return pd.DataFrame(records)

    except Exception as e:
        print(f"Error parseando SDMX: {e}")
        return pd.DataFrame()

## 📥 Extracción de Datos

Se extraen **5 indicadores** de Eurostat, cada uno genera 3 tablas (España, UE27, Ranking).

In [3]:
# Configuración común para todas las peticiones
params_eu = {"format": "JSON", "lang": "en", "detail": "full"}
print("🌍 Iniciando extracción de datos Eurostat...")

🌍 Iniciando extracción de datos Eurostat...


### 1. Brecha de Pobreza (sdg_10_30)

In [4]:
url_gap = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_10_30"
response_gap = requests.get(url_gap, params=params_eu)
df_gap_todos = parsear_eurostat_sdmx(
    response_gap.json(), "Brecha_Pobreza_%", filter_geo=None
)

if not df_gap_todos.empty:
    df_gap_todos = df_gap_todos[
        (df_gap_todos["Anio"] >= 2015) & (df_gap_todos["Anio"] <= 2024)
    ].copy()
    df_brecha_pobreza = df_gap_todos[df_gap_todos["geo_code"] == "ES"].copy()
    df_brecha_ue27 = df_gap_todos[df_gap_todos["geo_code"] == "EU27_2020"].copy()
    print(
        f"✅ Brecha - ES: {len(df_brecha_pobreza)} | UE27: {len(df_brecha_ue27)} | Ranking: {len(df_gap_todos)}"
    )
else:
    df_brecha_pobreza = pd.DataFrame()
    df_brecha_ue27 = pd.DataFrame()
    print("⚠️ Brecha: Sin datos")

✅ Brecha - ES: 10 | UE27: 10 | Ranking: 378


### 2. Tasa AROP (ilc_li02)

In [5]:
url_arop = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_li02"
response_arop = requests.get(url_arop, params=params_eu)
df_arop_eu_todos = parsear_eurostat_sdmx(
    response_arop.json(),
    "AROP_%",
    filter_geo=None,
    filter_unit="PC",
    filter_indic="LI_R_MD60",
    filter_age="TOTAL",
    filter_sex="T",
)

if not df_arop_eu_todos.empty:
    df_arop_eu_todos = df_arop_eu_todos[
        (df_arop_eu_todos["Anio"] >= 2015) & (df_arop_eu_todos["Anio"] <= 2024)
    ].copy()
    df_arop_eu = df_arop_eu_todos[df_arop_eu_todos["geo_code"] == "ES"].copy()
    df_arop_ue27 = df_arop_eu_todos[df_arop_eu_todos["geo_code"] == "EU27_2020"].copy()
    print(
        f"✅ AROP - ES: {len(df_arop_eu)} | UE27: {len(df_arop_ue27)} | Ranking: {len(df_arop_eu_todos)}"
    )
else:
    df_arop_eu = pd.DataFrame()
    df_arop_ue27 = pd.DataFrame()
    print("⚠️ AROP: Sin datos")

✅ AROP - ES: 10 | UE27: 10 | Ranking: 397


### 3. Gini (ilc_di12)

In [6]:
url_gini = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_di12"
response_gini = requests.get(url_gini, params=params_eu)
df_gini_todos = parsear_eurostat_sdmx(
    response_gini.json(),
    "Gini",
    filter_geo=None,
    filter_unit="PC",
    filter_age="TOTAL",
    filter_sex="T",
)

if not df_gini_todos.empty:
    df_gini_todos = df_gini_todos[
        (df_gini_todos["Anio"] >= 2015) & (df_gini_todos["Anio"] <= 2024)
    ].copy()
    df_gini_eu = df_gini_todos[df_gini_todos["geo_code"] == "ES"].copy()
    df_gini_ue27 = df_gini_todos[df_gini_todos["geo_code"] == "EU27_2020"].copy()
    print(
        f"✅ Gini - ES: {len(df_gini_eu)} | UE27: {len(df_gini_ue27)} | Ranking: {len(df_gini_todos)}"
    )
else:
    df_gini_eu = pd.DataFrame()
    df_gini_ue27 = pd.DataFrame()
    print("⚠️ Gini: Sin datos")

✅ Gini - ES: 10 | UE27: 10 | Ranking: 365


### 4. S80/S20 (ilc_di11)

In [7]:
url_s80s20 = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ilc_di11"
response_s80s20 = requests.get(url_s80s20, params=params_eu)
df_s80s20_todos = parsear_eurostat_sdmx(
    response_s80s20.json(),
    "S80S20_Ratio",
    filter_geo=None,
    filter_unit="RAT",
    filter_age="TOTAL",
    filter_sex="T",
)

if not df_s80s20_todos.empty:
    df_s80s20_todos = df_s80s20_todos[
        (df_s80s20_todos["Anio"] >= 2015) & (df_s80s20_todos["Anio"] <= 2024)
    ].copy()
    df_s80s20_eu = df_s80s20_todos[df_s80s20_todos["geo_code"] == "ES"].copy()
    df_s80s20_ue27 = df_s80s20_todos[df_s80s20_todos["geo_code"] == "EU27_2020"].copy()
    print(
        f"✅ S80/S20 - ES: {len(df_s80s20_eu)} | UE27: {len(df_s80s20_ue27)} | Ranking: {len(df_s80s20_todos)}"
    )
else:
    df_s80s20_eu = pd.DataFrame()
    df_s80s20_ue27 = pd.DataFrame()
    print("⚠️ S80/S20: Sin datos")

✅ S80/S20 - ES: 10 | UE27: 10 | Ranking: 400


### 5. Impacto Redistributivo (ilc_di12b, ilc_di12c)

In [8]:
# España - Construir desde cero sin heredar de df_gini_eu
df_impacto_redistrib_es = pd.DataFrame()

for code, col in [
    ("ilc_di12b", "Gini_Antes_SinPensiones"),
    ("ilc_di12c", "Gini_Antes_ConPensiones"),
    ("ilc_di12", "Gini_Despues"),
]:
    url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{code}"
    r = requests.get(url, params=params_eu)
    if r.status_code == 200:
        df_temp = parsear_eurostat_sdmx(
            r.json(),
            col,
            filter_geo="ES",
            filter_unit="PC",
            filter_age="TOTAL",
            filter_sex="T",
        )

        # Seleccionar solo columnas relevantes
        cols_necesarias = ["geo_code", "geo_name", "Anio", col]
        df_temp_clean = df_temp[
            [c for c in cols_necesarias if c in df_temp.columns]
        ].copy()

        if df_impacto_redistrib_es.empty:
            df_impacto_redistrib_es = df_temp_clean
        else:
            # Merge por geo_code, geo_name y Anio para mantener la geografía
            merge_cols = [
                c
                for c in ["geo_code", "geo_name", "Anio"]
                if c in df_impacto_redistrib_es.columns and c in df_temp_clean.columns
            ]
            df_impacto_redistrib_es = pd.merge(
                df_impacto_redistrib_es, df_temp_clean, on=merge_cols, how="outer"
            )

df_impacto_redistrib_es = df_impacto_redistrib_es.sort_values("Anio").reset_index(
    drop=True
)

# UE27 - Construir desde cero sin heredar de df_gini_ue27
df_impacto_redistrib_ue27 = pd.DataFrame()

for code, col in [
    ("ilc_di12b", "Gini_Antes_SinPensiones"),
    ("ilc_di12c", "Gini_Antes_ConPensiones"),
    ("ilc_di12", "Gini_Despues"),
]:
    url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{code}"
    r = requests.get(url, params=params_eu)
    if r.status_code == 200:
        df_temp = parsear_eurostat_sdmx(
            r.json(),
            col,
            filter_geo=None,
            filter_unit="PC",
            filter_age="TOTAL",
            filter_sex="T",
        )

        # Filtrar UE27
        df_temp = df_temp[df_temp["geo_code"] == "EU27_2020"].copy()

        # Seleccionar solo columnas relevantes
        cols_necesarias = ["geo_code", "geo_name", "Anio", col]
        df_temp_clean = df_temp[
            [c for c in cols_necesarias if c in df_temp.columns]
        ].copy()

        if df_impacto_redistrib_ue27.empty:
            df_impacto_redistrib_ue27 = df_temp_clean
        else:
            # Merge por geo_code, geo_name y Anio para mantener la geografía
            merge_cols = [
                c
                for c in ["geo_code", "geo_name", "Anio"]
                if c in df_impacto_redistrib_ue27.columns and c in df_temp_clean.columns
            ]
            df_impacto_redistrib_ue27 = pd.merge(
                df_impacto_redistrib_ue27, df_temp_clean, on=merge_cols, how="outer"
            )

df_impacto_redistrib_ue27 = df_impacto_redistrib_ue27.sort_values("Anio").reset_index(
    drop=True
)

print(
    f"✅ Impacto Redistributivo - ES: {len(df_impacto_redistrib_es)} | UE27: {len(df_impacto_redistrib_ue27)}"
)

✅ Impacto Redistributivo - ES: 21 | UE27: 15


### 6. Limpieza columnas debug

In [9]:
# Limpiar columnas age/sex de todos los DataFrames
dataframes_limpiar = [
    ("df_gap_todos", df_gap_todos),
    ("df_arop_eu_todos", df_arop_eu_todos),
    ("df_gini_todos", df_gini_todos),
    ("df_s80s20_todos", df_s80s20_todos),
    ("df_impacto_redistrib_es", df_impacto_redistrib_es),
    ("df_impacto_redistrib_ue27", df_impacto_redistrib_ue27),
]

for df_name, df in dataframes_limpiar:
    cols_drop = [c for c in ["age", "age_label", "sex", "sex_label"] if c in df.columns]
    if cols_drop:
        globals()[df_name] = df.drop(columns=cols_drop)
        print(f"  ✅ {df_name}: eliminadas {cols_drop}")

print("\n✅ Limpieza completada: columnas age/sex eliminadas de todos los DataFrames")

  ✅ df_gap_todos: eliminadas ['age', 'age_label', 'sex', 'sex_label']
  ✅ df_arop_eu_todos: eliminadas ['age', 'age_label', 'sex', 'sex_label']
  ✅ df_gini_todos: eliminadas ['age', 'age_label']
  ✅ df_s80s20_todos: eliminadas ['age', 'age_label', 'sex', 'sex_label']

✅ Limpieza completada: columnas age/sex eliminadas de todos los DataFrames


### 💾 Guardar DataFrames en Pickle

In [10]:
print("💾 Guardando DataFrames Eurostat en pickle...")

# Diccionario con todos los DataFrames Eurostat
dataframes_eurostat = {
    "df_gini_es": df_gini_eu,
    "df_gini_ue27": df_gini_ue27,
    "df_gini_todos": df_gini_todos,
    "df_arop_es": df_arop_eu,
    "df_arop_ue27": df_arop_ue27,
    "df_arop_eu_todos": df_arop_eu_todos,
    "df_s80s20_es": df_s80s20_eu,
    "df_s80s20_ue27": df_s80s20_ue27,
    "df_s80s20_todos": df_s80s20_todos,
    "df_gap_es": df_brecha_pobreza,
    "df_gap_ue27": df_brecha_ue27,
    "df_gap_todos": df_gap_todos,
    "df_impacto_redistrib_es": df_impacto_redistrib_es,
    "df_impacto_redistrib_ue27": df_impacto_redistrib_ue27,
}

# Guardar cada DataFrame
for nombre, df in dataframes_eurostat.items():
    ruta = CACHE_DIR / f"{nombre}.pkl"
    with open(ruta, "wb") as f:
        pickle.dump(df, f)
    print(f"  ✅ {nombre}: {len(df)} registros → {ruta.name}")

print(f"\n✅ Total guardados: {len(dataframes_eurostat)} pickles Eurostat")

💾 Guardando DataFrames Eurostat en pickle...
  ✅ df_gini_es: 10 registros → df_gini_es.pkl
  ✅ df_gini_ue27: 10 registros → df_gini_ue27.pkl
  ✅ df_gini_todos: 365 registros → df_gini_todos.pkl
  ✅ df_arop_es: 10 registros → df_arop_es.pkl
  ✅ df_arop_ue27: 10 registros → df_arop_ue27.pkl
  ✅ df_arop_eu_todos: 397 registros → df_arop_eu_todos.pkl
  ✅ df_s80s20_es: 10 registros → df_s80s20_es.pkl
  ✅ df_s80s20_ue27: 10 registros → df_s80s20_ue27.pkl
  ✅ df_s80s20_todos: 400 registros → df_s80s20_todos.pkl
  ✅ df_gap_es: 10 registros → df_gap_es.pkl
  ✅ df_gap_ue27: 10 registros → df_gap_ue27.pkl
  ✅ df_gap_todos: 378 registros → df_gap_todos.pkl
  ✅ df_impacto_redistrib_es: 21 registros → df_impacto_redistrib_es.pkl
  ✅ df_impacto_redistrib_ue27: 15 registros → df_impacto_redistrib_ue27.pkl

✅ Total guardados: 14 pickles Eurostat
